In [ ]:
# Predict house prices: regression

In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

1.12.0


In [4]:
# The Boston housing prices dataset
boston_housing = keras.datasets.boston_housing

(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

# Shuffle the training set
order = np.argsort(np.random(train_labels.shape))
train_data = train_data[order]
train_labels = train_labels[order]

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/tf-keras-datasets/boston_housing.npz: None -- [Errno 60] Operation timed out